> Module: production<br>
> Assignment: 1<br>
> Name: Chun-Yuan Chen

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [2]:
# Write your code below.
%load_ext dotenv
%dotenv

In [3]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [4]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)

In [5]:
for path in parquet_files[:10]:
    print(path) # CYC: Just added this to make sure the paths are found correctly for 10 instances.

../../05_src/data/prices\ACN\ACN_2001\part.0.parquet
../../05_src/data/prices\ACN\ACN_2001\part.1.parquet
../../05_src/data/prices\ACN\ACN_2002\part.0.parquet
../../05_src/data/prices\ACN\ACN_2002\part.1.parquet
../../05_src/data/prices\ACN\ACN_2003\part.0.parquet
../../05_src/data/prices\ACN\ACN_2003\part.1.parquet
../../05_src/data/prices\ACN\ACN_2004\part.0.parquet
../../05_src/data/prices\ACN\ACN_2004\part.1.parquet
../../05_src/data/prices\ACN\ACN_2005\part.0.parquet
../../05_src/data/prices\ACN\ACN_2005\part.1.parquet


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [6]:
# Write your code below.
dd_px = dd.read_parquet(parquet_files).set_index("ticker", sorted=True)
dd_px = dd_px.map_partitions(lambda df: df.sort_values(['Date']))
dd_shift = dd_px.copy() # CYC: Just made a copy for later, just to be safe.

In [7]:
# CYC: Created a new variable called "Close_lag_1" to store the one-day lag of the variable "Close".
dd_shift = dd_shift.groupby('ticker', group_keys=False).apply(
    lambda x: x.sort_values('Date').assign(Close_lag_1 = x['Close'].shift(1))
)

In [8]:
# CYC: Created a new variable called "Adj Close_lag_1" to store the one-day lag of the variable "Adj Close".
dd_shift = dd_shift.groupby('ticker', group_keys=False).apply(
    lambda x: x.sort_values('Date').assign(**{'Adj Close_lag_1': x['Adj Close'].shift(1)})
)

In [9]:
# CYC: Created a new variable called "returns" based on "Close" and "Close_lag_1".
dd_shift["returns"] = (dd_shift["Close"] / dd_shift["Close_lag_1"]) - 1

In [10]:
# CYC: Created a new variable called "hi_lo_range" for each day's High minus Low.
dd_shift["hi_lo_range"] = dd_shift["High"] - dd_shift["Low"]

In [11]:
# CYC: Assigned to `dd_feat`for checking the structure.
dd_feat = dd_shift.copy()

In [12]:
dd_feat # CYC: Take a look this framework of data in advance. Next line will show the results filled in.

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj Close_lag_1,returns,hi_lo_range
npartitions=3061,,,,,,,,,,,,,
ACN,datetime64[ns],float64,float64,float64,float64,float64,float64,string,int32,float64,float64,float64,float64
ACN,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...,...


In [13]:
dd_feat.compute()

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj Close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,,
ACN,2001-07-19,15.10,15.29,15.00,15.17,11.404394,34994300.0,ACN.csv,2001,NaN,NaN,NaN,0.29
ACN,2001-07-20,15.05,15.05,14.80,15.01,11.284108,9238500.0,ACN.csv,2001,15.17,11.404394,-0.010547,0.25
ACN,2001-07-23,15.00,15.01,14.55,15.00,11.276587,7501000.0,ACN.csv,2001,15.01,11.284108,-0.000666,0.46
ACN,2001-07-24,14.95,14.97,14.70,14.86,11.171341,3537300.0,ACN.csv,2001,15.00,11.276587,-0.009333,0.27
ACN,2001-07-25,14.70,14.95,14.65,14.95,11.238999,4208100.0,ACN.csv,2001,14.86,11.171341,0.006057,0.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,2020-03-26,4.06,4.53,3.88,4.51,4.510000,1668500.0,ZIXI.csv,2020,4.00,4.000000,0.127500,0.65
ZIXI,2020-03-27,4.49,4.71,4.10,4.60,4.600000,1146800.0,ZIXI.csv,2020,4.51,4.510000,0.019956,0.61
ZIXI,2020-03-30,4.83,4.87,4.44,4.64,4.640000,1212000.0,ZIXI.csv,2020,4.60,4.600000,0.008696,0.43


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [14]:
# Write your code below.
dd_feat = dd_feat.compute()
dd_feat[['Date', 'Close', 'Close_lag_1', 'returns', 'Adj Close', 'Adj Close_lag_1', 'High', 'Low', 'hi_lo_range']] # CYC: I added this line to peek at the computed results of interest in dd_feat, and make sure all variables derived correctly. 

,Date,Close,Close_lag_1,returns,Adj Close,Adj Close_lag_1,High,Low,hi_lo_range
ticker,,,,,,,,,
ACN,2001-07-19,15.17,NaN,NaN,11.404394,NaN,15.29,15.00,0.29
ACN,2001-07-20,15.01,15.17,-0.010547,11.284108,11.404394,15.05,14.80,0.25
ACN,2001-07-23,15.00,15.01,-0.000666,11.276587,11.284108,15.01,14.55,0.46
ACN,2001-07-24,14.86,15.00,-0.009333,11.171341,11.276587,14.97,14.70,0.27
ACN,2001-07-25,14.95,14.86,0.006057,11.238999,11.171341,14.95,14.65,0.30
...,...,...,...,...,...,...,...,...,...
ZIXI,2020-03-26,4.51,4.00,0.127500,4.510000,4.000000,4.53,3.88,0.65
ZIXI,2020-03-27,4.60,4.51,0.019956,4.600000,4.510000,4.71,4.10,0.61
ZIXI,2020-03-30,4.64,4.60,0.008696,4.640000,4.600000,4.87,4.44,0.43


In [15]:
# CYC: I added these for later use.
import numpy as np 
import pandas as pd
dd_feat['group'] = dd_feat.index
pd_feat = dd_feat.copy()

In [16]:
pd_feat["Date"] = pd.to_datetime(pd_feat["Date"]) # CYC: I added this to ensure "Date" is datetime.
pd_feat = pd_feat.sort_values(["group", "Date"])

In [17]:
# CYC: This chunk of code for creating the variable "returns_roll_10days" for deriving moving average of `returns` using a window of 10 days.
# CYC: Since the dates aren't always continuous, I used actual calendar days instead of a fixed 10-row window.
# CYC: There could have different ways to define a 10-day window. Here, I'm including the current day and tracing back the previous 9 calendar days.
pd_feat["returns_roll_10days"] = (
    pd_feat.groupby("group").apply(
        lambda a: a.assign(
            returns_roll_10days=a["Date"].map(
                lambda b: a.loc[(a["Date"] >= b - pd.Timedelta(days=9)) & (a["Date"] <= b), "returns"].mean()
            )
        )
    )["returns_roll_10days"].values
)

In [18]:
# CYC: Just picked a ticker (ACN) to check if returns_roll_10days is working as expected.
# CYC: On 2001-07-19, this is the 1st record (sorted by Date) for this ticker. No ealier record, so returns and returns_roll_10days aren't calculated.
# CYC: On 2001-07-20, it's the 2nd record. No ealier returns, so returns_roll_10days equals the current return.
# CYC: On 2001-07-25, it's the 5th record (after 2001-07-16), with only 4 return values available (including itself), so returns_roll_10days is the average of those.
# CYC: For most other dates, e.g., 2001-07-30, the window spans from 2001-07-21 to 2001-07-30 (my 10-day definition). All valid returns in that range are averaged to compute returns_roll_10days.
pd_feat.loc[pd_feat['group'] == 'ACN', ['Date', 'Close', 'Close_lag_1', 'Adj Close', 'Adj Close_lag_1', 'High', 'Low', 'hi_lo_range', 'returns', 'returns_roll_10days']]

,Date,Close,Close_lag_1,Adj Close,Adj Close_lag_1,High,Low,hi_lo_range,returns,returns_roll_10days
ticker,,,,,,,,,,
ACN,2001-07-19,15.170000,NaN,11.404394,NaN,15.290000,15.000000,0.290000,NaN,NaN
ACN,2001-07-20,15.010000,15.170000,11.284108,11.404394,15.050000,14.800000,0.250000,-0.010547,-0.010547
ACN,2001-07-23,15.000000,15.010000,11.276587,11.284108,15.010000,14.550000,0.460000,-0.000666,-0.005607
ACN,2001-07-24,14.860000,15.000000,11.171341,11.276587,14.970000,14.700000,0.270000,-0.009333,-0.006849
ACN,2001-07-25,14.950000,14.860000,11.238999,11.171341,14.950000,14.650000,0.300000,0.006057,-0.003623
...,...,...,...,...,...,...,...,...,...,...
ACN,2020-03-26,171.339996,151.820007,171.339996,151.820007,171.600006,150.000000,21.600006,0.128573,0.016666
ACN,2020-03-27,162.880005,171.339996,162.880005,171.339996,169.289993,160.610001,8.679993,-0.049375,0.006257
ACN,2020-03-30,169.820007,162.880005,169.820007,162.880005,170.369995,163.779999,6.589996,0.042608,0.023273


In [19]:
pd_feat.to_excel('pd_feat.xlsx', index=True) # CYC: Just exported the data to eyeball a few tickers and make sure things are working fine, as a check spot as well.

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
> Response: I would simply say if the dataset is not so huge to fit in memory, switching to pandas makes calculations way easier, especially for it involving some complex grouping, sorting, and averaging processes. I have noticed that if not careful, Dask form can easily cause misalignment issues.

+ Would it have been better to do it in Dask? Why?
> Response: I would say maybe better if the dataset was too big to fit in memory, Dask would split the data into partitions and runs calculations in parallel. In this condition, Dask performs in a quicker way than Pandas form. 

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.